In [1]:
# pip install watchdog

In [2]:
import sys
import time
import logging

from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import FileSystemEventHandler

from IPython.display import display
from IPython.display import clear_output
from ipywidgets import IntProgress

import json

import pandas as pd

from os import listdir
from os.path import isfile, join

In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 20;

<IPython.core.display.Javascript object>

In [4]:
df = pd.DataFrame(columns=['name','neg', 'neu', 'pos', 'compound'])

path = '../responses/'

In [5]:
def insertDataFromJSON(filepath):
    global df
    with open(filepath) as json_file:
        json_data = json.load(json_file)

        df = df.append({
            'name': json_data['name'], 
            'neg': json_data['emotions']['neg'],
            'neu': json_data['emotions']['neu'],
            'pos': json_data['emotions']['pos'],
            'compound': json_data['emotions']['compound']
        }, ignore_index=True)

In [6]:
### observe new files
def on_file_created(filepath):
    global df
    
    if(not filepath.endswith(".json")):
        print(df)
        return
    
    insertDataFromJSON(filepath)
    
    df = df.sort_values(by=['compound'], ascending=False).reset_index(drop=True)

    mean_text = 'Mean: {}'.format(round(df["compound"].mean(),3))
    display(mean_text)
    display(df)

In [7]:
def observe(path = "./"):

    files_to_be_created = 100

    pb = IntProgress(description='Iter 0/{}'.format(files_to_be_created), min=0, max=files_to_be_created)
    display(pb)

    class EventHandler(FileSystemEventHandler):
        def on_created(self, event):
            pb.value += 1
            pb.description = 'Iter {}/{}'.format(pb.value, files_to_be_created)
            clear_output(wait=True)
            display(pb)
            on_file_created(event.src_path)

    event_handler = EventHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=True)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

In [8]:
df.drop(df.index, inplace=True)

### read existing files

for filename in [f for f in listdir(path) if isfile(join(path, f))]:
    
    if(not filename.endswith(".json")):
        continue
    
    insertDataFromJSON(join(path, filename))
    
df = df.sort_values(by=['compound'], ascending=False).reset_index(drop=True)
mean_text = 'Mean: {}'.format(round(df["compound"].mean(),3))

display(mean_text)
display(df)
        
### observe

observe('../responses/')

'Mean: 0.519'

,name,neg,neu,pos,compound
0,Page #7,0.533,0.148,0.981,0.941
1,Page #6,0.994,0.980,0.901,0.824
2,Page #8,0.366,0.119,0.211,0.798
3,Page #3,0.010,0.993,0.520,0.610
4,Page #2,0.585,0.510,0.815,0.593
5,Page #4,0.131,0.756,0.764,0.589
6,Page #9,0.503,0.711,0.257,0.426
7,Page #9,0.503,0.711,0.257,0.426
8,Page #1,0.019,0.232,0.261,0.344
9,Page #5,0.778,0.866,0.049,0.152


IntProgress(value=0, description='Iter 0/100')